In [1]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
import os
import asyncio
import random
from playwright.async_api import async_playwright
from typing import List, Dict, Any, Literal
from fake_useragent import UserAgent
import time
from functools import reduce

In [9]:
ua = UserAgent()

In [166]:
import os

In [41]:
async def set_chromium(playwright, headless=True):
    return await playwright.chromium.launch(
        headless=headless,
        args=[
            "--disable-blink-features=AutomationControlled",
            "--disable-features=IsolateOrigins,site-per-process",
            "--disable-dev-shm-usage",
            "--no-sandbox",
            "--disable-setuid-sandbox",
        ],
    )


async def set_firefox(playwright, headless=True):
    return await playwright.firefox.launch(
        headless=headless,
        firefox_user_prefs={
            "dom.webdriver.enabled": False,
            "privacy.resistFingerprinting": False,
            "browser.cache.disk.enable": True,
            "browser.cache.memory.enable": True,
        },
        args=["--disable-dev-shm-usage", "--no-sandbox"],
    )


async def set_browser(
    playwright, engine: Literal["firefox", "chromium", "random"], headless: bool = True
):
    if engine == "random":
        engine = random.choice(["chromium", "firefox"])

    match engine:
        case "firefox":
            return await set_firefox(playwright, headless=headless)
        case "chromium":
            return await set_chromium(playwright, headless=headless)
        case _:
            raise ValueError(f"Engine {_} not recognized.")


async def set_context(browser):
    # Create a new browser context with random viewport size
    viewport_width = random.randint(1000, 1920)
    viewport_height = random.randint(800, 1080)

    return await browser.new_context(
        user_agent=ua.random,
        viewport={"width": viewport_width, "height": viewport_height},
        locale=random.choice(["pt-BR", "pt-PT", "es-AR", "es-UY", "es-PY"]),
        timezone_id=random.choice(
            [
                "America/Sao_Paulo",
                "America/Rio_Branco",
                "America/Manaus",
                "America/Fortaleza",
                "America/Recife",
                "America/Argentina/Buenos_Aires",
                "America/Montevideo",
            ]
        ),
        permissions=["geolocation"],
        has_touch=random.choice([True, False]),
    )


async def set_page(context):
    page = await context.new_page()

    # Emulate human-like behavior by intercepting WebDriver calls
    await page.add_init_script("""
    Object.defineProperty(navigator, 'webdriver', {
        get: () => false
    });
    
    // Add plugins length
    Object.defineProperty(navigator, 'plugins', {
        get: () => [1, 2, 3, 4, 5]
    });
    
    // Overwrite the languages property
    Object.defineProperty(navigator, 'languages', {
        get: () => ['pt-BR', 'pt']
    });
    """)

    return page

In [108]:
async def do_movements(page):
    # Random scrolling behavior
    for _ in range(random.randint(3, 7)):
        await page.mouse.wheel(0, random.randint(100, 500))
        await asyncio.sleep(random.uniform(0.1, 0.5))

    # Random mouse movements
    for _ in range(random.randint(3, 8)):
        await page.mouse.move(
            random.randint(100, 1000 - 100),
            random.randint(100, 800 - 100),
            steps=random.randint(1, 4),
        )
        await asyncio.sleep(random.uniform(0.05, 0.1))

In [96]:
async def clear_headers(original_headers):
    """Limpa os headers mantendo apenas os campos necessários."""
    # Campos que consideramos necessários para a requisição
    necessary_fields = [
        "user-agent",
        "accept",
        "referer",
        "authorization",
        "origin",
        "host",
    ]

    # Cria um novo dicionário apenas com os campos necessários
    cleaned_headers = {
        k: v for k, v in original_headers.items() if k.lower() in necessary_fields
    }

    return cleaned_headers


async def perform_api_request(context, headers, endpoint):
    """Realiza uma requisição GET para o endpoint usando os headers fornecidos."""
    cleaned_headers = await clear_headers(headers)
    api_page = await set_page(context)
    try:
        response = await api_page.request.get(endpoint, headers=cleaned_headers)
        if response.ok:
            return await response.json()

        error_text = await response.text()
        print(f"Falha na requisição. Status: {response.status} | Erro: {error_text}")
        return None

    except Exception as e:
        print(f"Erro ao fazer a requisição: {str(e)}")
        return None
    finally:
        # Fecha a página da API
        await api_page.close()

# Obter base da Betha

In [106]:
def generate_endpoint(limit, offset):
    return f"https://api.transparencia.betha.cloud/transparencia/auth/portais?filter=nome+like+%27%2525%2525%27&limit={limit}&offset={offset}"

In [109]:
import asyncio
from playwright.async_api import async_playwright
from typing import List, Dict, Any

url = "https://transparencia.betha.cloud/"


def generate_endpoint(limit, offset):
    return f"https://api.transparencia.betha.cloud/transparencia/auth/portais?filter=nome+like+%27%2525%2525%27&limit={limit}&offset={offset}"


responses_list: List[Dict[str, Any]] = []


def handle_response(response):
    condition = (
        (response.status == 200)
        & (response.url[-3:] != ".js")
        & (response.url[-4:] != ".css")
        & (response.url[-4:] != ".ttf")
        & (response.url[-4:] != ".svg")
        & (response.url[-4:] != ".png")
        & (response.url[-4:] != ".jpg")
        & (".js?" not in response.url)
        & (".woff" not in response.url)
    )

    if condition:
        responses_list.append(response)


async with async_playwright() as playwright:
    browser = await set_browser(playwright, engine="random")
    context = await set_context(browser)
    page = await set_page(context)

    page.on("response", handle_response)

    await page.goto(url, wait_until="networkidle")

    # await do_movements(page)

    portais_request = [
        resp
        for resp in responses_list
        if (("/portais" in resp.url) and ("estados" not in resp.url))
    ][0]
    portais_json = await portais_request.json()
    total_portais = portais_json.get("total", 2000)

    request_headers = await portais_request.request.all_headers()

    batch_size = 100

    downloaded_results = []
    for offset in range(0, total_portais, batch_size):
        end = min(total_portais, offset + batch_size)
        print(f"Downloading from {offset} to {end}")
        limit = end - offset
        endpoint = generate_endpoint(limit, offset)
        result = await perform_api_request(context, request_headers, endpoint)
        downloaded_results.append(result)
        await do_movements(page)

    await browser.close()


portais = reduce(lambda x, y: x + y, [x["content"] for x in downloaded_results])

In [126]:
import json

base_url = "https://transparencia.betha.cloud"


for portal in portais:
    if _hash := portal.get("hash"):
        portal.update(
            {
                "homepage": f"{base_url}/#/{_hash}",
            }
        )

with open("portais.json", "w") as f:
    f.write(json.dumps(portais, indent=4))

In [127]:
portais[26]

{'id': 1053,
 'nome': 'CÂMARA MUNICIPAL ALFREDO WAGNER',
 'estado': 'Santa Catarina',
 'uf': 'SC',
 'codigoIbge': 4200705,
 'cor': 'AZUL',
 'portalEntidades': [{'id': 2100, 'database': 1244, 'entidade': 2363}],
 'createdBy': 'eduarda.diniz',
 'createdIn': '2024-04-26T11:44:51.08',
 'updatedBy': 'julianahasckel',
 'updatedIn': '2024-07-13T17:04:33.533',
 'municipio': 'Alfredo Wagner',
 'textoAlternativoImagemFundo': None,
 'seloS3Key': None,
 'publicarSelo': False,
 'hash': 'UnsG5XuuoBixbrD6_V-FpA==',
 'homepage': 'https://transparencia.betha.cloud/#/UnsG5XuuoBixbrD6_V-FpA=='}

In [148]:
!pip install pandas -qqq

In [149]:
import pandas as pd

idxs = pd.Series([portal["id"] for portal in portais])

In [152]:
idxs.nunique()

1090

# Baixar Menus

In [153]:
!pip install tenacity -qqq

In [154]:
class MenuRequestNotFoundError(Exception):
    """Exceção customizada para quando a requisição do menu não é encontrada."""

    pass


class JsonRequestError(Exception):
    """Exceção customizada para quando há erro ao obter o JSON."""

    pass

In [174]:
from playwright.async_api import TimeoutError as PlaywrightTimeoutError
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
)


# Utilizando tenacity para implementar retries
@retry(
    stop=stop_after_attempt(3),  # Tenta no máximo 3 vezes
    wait=wait_exponential(
        multiplier=1, min=1, max=10
    ),  # Espera entre tentativas aumenta exponencialmente
    retry=retry_if_exception_type(
        (PlaywrightTimeoutError, MenuRequestNotFoundError, JsonRequestError)
    ),
    reraise=True,
)
async def get_menu_info(url: str, count: int) -> List[Dict] | None:
    """
    Extrai informações de menu de uma URL com sistema de retries.

    Args:
        url: URL a ser processada
        count: Contador do loop atual (usado para geração de ID)

    Returns:
        Lista de dicionários contendo informações de menu ou None em caso de falha
    """
    print(f"\n[{count + 1}] Processando URL: {url}")
    responses_list = []

    def handle_response(response):
        condition = (
            (response.status == 200)
            and (response.url[-3:] != ".js")
            and (response.url[-4:] != ".css")
            and (response.url[-4:] != ".ttf")
            and (response.url[-4:] != ".svg")
            and (response.url[-4:] != ".png")
            and (response.url[-4:] != ".jpg")
            and (".js?" not in response.url)
            and (".woff" not in response.url)
        )

        if condition:
            responses_list.append(response)

    try:
        async with async_playwright() as playwright:
            browser = await set_browser(playwright, engine="random", headless=True)
            context = await set_context(browser)
            page = await set_page(context)

            page.on("response", handle_response)
            await page.goto(url, wait_until="networkidle", timeout=60000)
            await do_movements(page)
            menu_requests = [resp for resp in responses_list if "/menu" in resp.url]

            if not menu_requests:
                print(
                    f"  ERRO: Requisição de menu não encontrada. Total de respostas: {len(responses_list)}"
                )
                captured_urls = [resp.url for resp in responses_list]
                print(f"  URLs capturadas: {captured_urls[:5]}...")
                raise MenuRequestNotFoundError("Requisição de menu não encontrada")

            menu_request = menu_requests[0]

            try:
                menu_json = await menu_request.json()
                if not isinstance(menu_json, list) or len(menu_json) == 0:
                    raise JsonRequestError("Dados JSON inválidos ou vazios")

            except Exception as e:
                print(f"  ERRO ao processar JSON: {str(e)}")
                body_text = await menu_request.text()
                print(
                    f"  Corpo da resposta (primeiros 100 caracteres): {body_text[:100]}..."
                )
                raise JsonRequestError(f"Erro ao processar JSON: {str(e)}")

            finally:
                await browser.close()

            return menu_json

    except (PlaywrightTimeoutError, MenuRequestNotFoundError, JsonRequestError) as e:
        print(f"  ERRO durante a tentativa: {str(e)}. Tentando novamente...")
        raise  # Relanço a exceção para que o retry funcione

    except Exception as e:
        print(f"  ERRO inesperado: {str(e)}")
        return None

In [175]:
async def save_to_json(data: Dict, filename: str):
    """
    Salva ou atualiza os dados no arquivo JSON.

    Args:
        data: Dicionário com os dados a serem salvos
        filename: Nome do arquivo JSON
    """
    try:
        # Verificar se o arquivo já existe
        if os.path.exists(filename):
            # Carregar o conteúdo existente
            with open(filename, "r", encoding="utf-8") as f:
                try:
                    existing_data = json.load(f)
                except json.JSONDecodeError:
                    print(
                        f"Erro ao carregar o arquivo JSON existente. Criando novo arquivo."
                    )
                    existing_data = {}
        else:
            existing_data = {}

        # Atualizar com os novos dados
        existing_data.update(data)

        # Salvar o arquivo atualizado
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=2)

    except Exception as e:
        print(f"ERRO ao salvar dados no arquivo JSON: {str(e)}")

In [176]:
async def process_urls(list_urls: List[str], output_file: str = "menu_data.json"):
    """
    Processa uma lista de URLs e salva os resultados.

    Args:
        list_urls: Lista de URLs a serem processadas
        output_file: Nome do arquivo de saída
    """
    total_urls = len(list_urls)
    processed = 0
    failed = 0

    print(f"Iniciando processamento de {total_urls} URLs...")

    for count, url in enumerate(list_urls):
        try:
            # Tenta obter as informações do menu
            menu_info = await get_menu_info(url, count)

            if menu_info:
                # Verificar e definir o ID do portal
                portal_ids = [
                    item.get("portal") for item in menu_info if "portal" in item
                ]

                if portal_ids and portal_ids[0] is not None:
                    idx = portal_ids[0]  # Usa o primeiro ID de portal encontrado
                else:
                    idx = 2000 + count  # Gera um ID com base no contador
                    # Adiciona o ID do portal a todos os itens no menu_info que não têm
                    for item in menu_info:
                        if "portal" not in item:
                            item["portal"] = idx

                # Preparar os dados para salvar
                data = {
                    str(idx): menu_info
                }  # Converte o idx para string para ser chave no JSON

                # Salvar os dados
                await save_to_json(data, output_file)

                processed += 1
                print(f"[{count + 1}/{total_urls}] Sucesso! ID do portal: {idx}")
            else:
                failed += 1
                print(f"[{count + 1}/{total_urls}] Falha ao processar URL: {url}")

        except Exception as e:
            failed += 1
            print(f"[{count + 1}/{total_urls}] Erro ao processar URL {url}: {str(e)}")

        # Adicionar uma pausa aleatória entre requisições para evitar detecção
        await asyncio.sleep(random.uniform(1, 3))

    print(f"\nProcessamento concluído!")
    print(f"Total de URLs: {total_urls}")
    print(f"Processadas com sucesso: {processed}")
    print(f"Falhas: {failed}")

In [173]:
list_urls = [x["homepage"] for x in portais]

output_file = "dados-menus-betha.json"

await process_urls(list_urls, output_file)

Iniciando processamento de 1090 URLs...

[1] Processando URL: https://transparencia.betha.cloud/#/zGMBIFjERq9GoD5qnYNaYQ==
[1/1090] Sucesso! ID do portal: 825

[2] Processando URL: https://transparencia.betha.cloud/#/SBT3IItGYwyL2QDNqpv59A==
[2/1090] Sucesso! ID do portal: 482

[3] Processando URL: https://transparencia.betha.cloud/#/qUIvAT_81FdXyxBi2eYEvw==
[3/1090] Sucesso! ID do portal: 385

[4] Processando URL: https://transparencia.betha.cloud/#/_wWlo-Vmj0VErKUHKggRIg==
  ERRO ao processar JSON: Dados JSON inválidos ou vazios
  Corpo da resposta (primeiros 100 caracteres): []...
  ERRO durante a tentativa: Erro ao processar JSON: Dados JSON inválidos ou vazios. Tentando novamente...

[4] Processando URL: https://transparencia.betha.cloud/#/_wWlo-Vmj0VErKUHKggRIg==
  ERRO ao processar JSON: Dados JSON inválidos ou vazios
  Corpo da resposta (primeiros 100 caracteres): []...
  ERRO durante a tentativa: Erro ao processar JSON: Dados JSON inválidos ou vazios. Tentando novamente...

[4

CancelledError: 

In [167]:
import os

In [139]:
import asyncio
from playwright.async_api import async_playwright
from typing import List, Dict, Any

url = "https://transparencia.betha.cloud/#/UnsG5XuuoBixbrD6_V-FpA=="


async def get_menu_info(url):
    responses_list: List[Dict[str, Any]] = []

    def handle_response(response):
        condition = (
            (response.status == 200)
            & (response.url[-3:] != ".js")
            & (response.url[-4:] != ".css")
            & (response.url[-4:] != ".ttf")
            & (response.url[-4:] != ".svg")
            & (response.url[-4:] != ".png")
            & (response.url[-4:] != ".jpg")
            & (".js?" not in response.url)
            & (".woff" not in response.url)
        )

        if condition:
            responses_list.append(response)

    async with async_playwright() as playwright:
        browser = await set_browser(playwright, engine="random")
        context = await set_context(browser)
        page = await set_page(context)

        page.on("response", handle_response)

        await page.goto(url, wait_until="networkidle")

        await do_movements(page)

        menu_request = [resp for resp in responses_list if "/menu" in resp.url][0]
        menu_json = await menu_request.json()

    return menu_json

In [140]:
url = "https://transparencia.betha.cloud/#/UnsG5XuuoBixbrD6_V-FpA=="
menu_info = await get_menu_info(url)

In [146]:
menu_info[1]

{'id': 56148,
 'nome': 'Servidores',
 'padrao': False,
 'ordem': 0,
 'icone': 'font-awesome-building-user tx__blue--l30',
 'descricao': 'Informações sobre os servidores públicos da Câmara',
 'portal': 1053,
 'consultas': [{'id': 97238,
   'titulo': 'Cargos e Vencimentos',
   'exibeDadosAbertos': True,
   'tipo': 'CONSULTA_PADRAO',
   'url': None},
  {'id': 97290,
   'titulo': 'Estagiários',
   'exibeDadosAbertos': True,
   'tipo': 'CONSULTA_PADRAO',
   'url': None},
  {'id': 97283,
   'titulo': 'Folha de Pagamento',
   'exibeDadosAbertos': False,
   'tipo': 'AGRUPADOR',
   'url': None},
  {'id': 97255,
   'titulo': 'Servidores Cedidos/Recebidos',
   'exibeDadosAbertos': False,
   'tipo': 'CONSULTA_PADRAO',
   'url': None},
  {'id': 97261,
   'titulo': 'Servidores e Remunerações',
   'exibeDadosAbertos': False,
   'tipo': 'CONSULTA_PADRAO',
   'url': None},
  {'id': 97250,
   'titulo': 'Servidores Públicos',
   'exibeDadosAbertos': False,
   'tipo': 'CONSULTA_PADRAO',
   'url': None},
 

In [129]:
responses_list

[<Response url='https://transparencia.betha.cloud/' request=<Request url='https://transparencia.betha.cloud/' method='GET'>>,
 <Response url='https://transparencia.betha.cloud/auth-callback.html' request=<Request url='https://transparencia.betha.cloud/auth-callback.html' method='GET'>>,
 <Response url='https://transparencia.betha.cloud/' request=<Request url='https://transparencia.betha.cloud/' method='GET'>>,
 <Response url='https://api.transparencia.betha.cloud/transparencia/api/portal' request=<Request url='https://api.transparencia.betha.cloud/transparencia/api/portal' method='GET'>>,
 <Response url='https://api.transparencia.betha.cloud/transparencia/api/menu' request=<Request url='https://api.transparencia.betha.cloud/transparencia/api/menu' method='GET'>>,
 <Response url='https://api.transparencia.betha.cloud/transparencia/api/portal/imagem-fundo' request=<Request url='https://api.transparencia.betha.cloud/transparencia/api/portal/imagem-fundo' method='GET'>>,
 <Response url='ht